#### SQL запросы для создания и редактирования таблиц в БД

In [ ]:
CREATE TABLE pair_history (
    token_1 TEXT NOT NULL,
	token_2 TEXT NOT NULL,
	open_time TIMESTAMPTZ,
    close_time TIMESTAMPTZ DEFAULT now(),
	side_1 TEXT NOT NULL,
	side_2 TEXT NOT NULL,
	qty_1 DOUBLE PRECISION NOT NULL,
	qty_2 DOUBLE PRECISION NOT NULL,
	open_price_1 DOUBLE PRECISION NOT NULL,
    open_price_2 DOUBLE PRECISION NOT NULL,
    close_price_1 DOUBLE PRECISION NOT NULL,
	close_price_2 DOUBLE PRECISION NOT NULL,
	fee_1 DOUBLE PRECISION NOT NULL,
	fee_2 DOUBLE PRECISION NOT NULL,
	leverage DOUBLE PRECISION NOT NULL,
    pnl_1 DOUBLE PRECISION NOT NULL,
	pnl_2 DOUBLE PRECISION NOT NULL,
    profit DOUBLE PRECISION NOT NULL
);

CREATE TABLE current_ob (
    token TEXT NOT NULL,
	ts BIGINT NOT NULL,
	update_time TIMESTAMPTZ NOT NULL,
	bid_price_0 DOUBLE PRECISION NOT NULL,
    bid_volume_0 DOUBLE PRECISION NOT NULL,
    bid_price_1 DOUBLE PRECISION NOT NULL,
    bid_volume_1 DOUBLE PRECISION NOT NULL,
	bid_price_2 DOUBLE PRECISION NOT NULL,
    bid_volume_2 DOUBLE PRECISION NOT NULL,
	bid_price_3 DOUBLE PRECISION NOT NULL,
    bid_volume_3 DOUBLE PRECISION NOT NULL,
	bid_price_4 DOUBLE PRECISION NOT NULL,
    bid_volume_4 DOUBLE PRECISION NOT NULL,
	ask_price_0 DOUBLE PRECISION NOT NULL,
    ask_volume_0 DOUBLE PRECISION NOT NULL,
	ask_price_1 DOUBLE PRECISION NOT NULL,
    ask_volume_1 DOUBLE PRECISION NOT NULL,
	ask_price_2 DOUBLE PRECISION NOT NULL,
    ask_volume_2 DOUBLE PRECISION NOT NULL,
	ask_price_3 DOUBLE PRECISION NOT NULL,
    ask_volume_3 DOUBLE PRECISION NOT NULL,
	ask_price_4 DOUBLE PRECISION NOT NULL,
    ask_volume_4 DOUBLE PRECISION NOT NULL,
	PRIMARY KEY (token, update_time)
);

CREATE TABLE IF NOT EXISTS tick_ob_test (
    token        TEXT       NOT NULL,
    time         TIMESTAMPTZ NOT NULL,
    bid_price    DOUBLE PRECISION NOT NULL,
    ask_price    DOUBLE PRECISION NOT NULL,
    bid_size     DOUBLE PRECISION NOT NULL,
    ask_size     DOUBLE PRECISION NOT NULL
);

#### PostgreSQL

In [1]:
from datetime import datetime, timedelta
from bot.core.db.postgres_manager import DBManager
from zoneinfo import ZoneInfo
from datetime import timedelta, timezone
from time import sleep
import pandas as pd
import polars as pl
from decimal import Decimal
from bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

# pl.Config.set_tbl_rows(50)
# pl.Config.set_trim_decimal_zeros(True)
# pl.Config.set_float_precision(5)

In [2]:
# db_manager.clear_table('market_data_5s')

In [3]:
all_ = db_manager.get_orderbooks()
all_tokens = all_['token'].unique().to_list()

In [4]:
tokens = []

for token in sorted(all_tokens):
    first_date = db_manager.get_oldest_date_in_orderbook(token)
    last_date = db_manager.get_orderbooks(token, interval='5m', start_date=datetime(2025, 11, 15)).tail(1)['time'][0]

    print(f'{token:>12}, {first_date}; {last_date}')
    
    # if first_date < datetime(2025, 10, 25, 12, tzinfo=ZoneInfo("Europe/Moscow")):
    #     print(f'{token:>12}, {first_date}; {last_date}')
    #     tokens.append(token)
    #     tokens.append(token)
    # elif last_date > datetime(2025, 10, 8, tzinfo=ZoneInfo("Europe/Moscow")):
    #     pass
    # else:
    

  1INCH_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
    ADA_USDT, 2025-11-09 11:53:25+03:00; 2025-11-21 14:30:00+03:00
   AERO_USDT, 2025-11-16 16:50:00+03:00; 2025-11-21 14:30:00+03:00
   ALGO_USDT, 2025-11-09 11:53:25+03:00; 2025-11-21 14:30:00+03:00
    APT_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
    ARB_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
   ARKM_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
   ATOM_USDT, 2025-11-09 11:53:25+03:00; 2025-11-21 14:30:00+03:00
   AVAX_USDT, 2025-11-16 16:50:00+03:00; 2025-11-21 14:30:00+03:00
   BLUR_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
  BRETT_USDT, 2025-11-16 16:50:00+03:00; 2025-11-21 14:30:00+03:00
   CELO_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
    CHZ_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
    CRV_USDT, 2025-10-11 12:00:00+03:00; 2025-11-21 14:30:00+03:00
   DOGE_USDT, 2025-11-09 11:53:25+03:00; 2025-11-21 14:30:00+0

In [ ]:
len(tokens)

In [22]:
db_manager.get_table('current_ob')

,token,update_ts,update_time,bid_price_0,bid_volume_0,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,...,ask_price_0,ask_volume_0,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,ask_price_4,ask_volume_4
0,IMX_USDT,1763307851,2025-11-16 18:44:11+03:00,0.371600,202.1,0.371500,1296.7,0.371400,4928.1,0.371300,...,0.37170,40.4,0.371800,2219.3,0.371900,3598.6,0.372000,4308.1,0.372100,3628.2
1,INJ_USDT,1763307851,2025-11-16 18:44:11+03:00,6.636000,24.5,6.635000,332.9,6.634000,474.7,6.633000,...,6.63700,67.2,6.638000,599.0,6.639000,499.2,6.640000,778.1,6.641000,610.6
2,IOTA_USDT,1763307851,2025-11-16 18:44:11+03:00,0.131000,6376.1,0.130900,90115.5,0.130800,84481.4,0.130700,...,0.13110,1350.6,0.131200,38447.9,0.131300,74342.4,0.131400,90690.9,0.131500,52039.8
3,IP_USDT,1763307851,2025-11-16 18:44:11+03:00,2.983500,33.5,2.983300,19.5,2.983000,5.8,2.982900,...,2.98380,1.9,2.984000,5.8,2.984100,9.9,2.984200,4.4,2.984300,19.1
4,JASMY_USDT,1763307851,2025-11-16 18:44:11+03:00,0.008539,879.0,0.008538,879.0,0.008537,61205.0,0.008536,...,0.00854,3329.0,0.008541,1870.0,0.008542,19981.0,0.008543,86823.0,0.008544,108540.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,TWT_USDT,1763307851,2025-11-16 18:44:11+03:00,1.140000,114.2,1.139900,189.0,1.139800,446.6,1.139700,...,1.14010,291.6,1.140200,42.5,1.140300,117.9,1.140400,203.9,1.140500,735.1
76,VET_USDT,1763307851,2025-11-16 18:44:11+03:00,0.015350,13429.0,0.015340,695998.0,0.015330,919120.0,0.015320,...,0.01536,98986.0,0.015370,420910.0,0.015380,422420.0,0.015390,428114.0,0.015400,168340.0
77,WCT_USDT,1763307851,2025-11-16 18:44:11+03:00,0.144400,18323.0,0.144300,26792.0,0.144200,34075.0,0.144100,...,0.14460,9403.0,0.144700,32859.0,0.144800,30650.0,0.144900,35223.0,0.145000,150054.0
78,WIF_USDT,1763307851,2025-11-16 18:44:11+03:00,0.414800,63543.0,0.414700,118088.0,0.414600,161861.0,0.414500,...,0.41490,1406.0,0.415000,104168.0,0.415100,231277.0,0.415200,187614.0,0.415300,165760.0


In [ ]:
add_pair_order(self, token_1, token_2, created_at, side_1, side_2, qty_1, qty_2,
                       price_1, price_2, usdt_1, usdt_2, rpnl_1, rpnl_2, upnl_1, upnl_2,
                       leverage, profit_1, profit_2, profit, status)

In [12]:
current_pairs = db_manager.get_table('pairs', df_type='polars')
current_pairs

token_1,token_2,created_at,side_1,side_2,qty_1,qty_2,open_price_1,open_price_2,usdt_1,usdt_2,leverage,rpnl_1,rpnl_2,upnl_1,upnl_2,profit_1,profit_2,profit,status,mode
str,str,"datetime[μs, Europe/Moscow]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""XRP_USDT""","""LDO_USDT""",2025-11-15 21:20:25 MSK,"""long""","""short""",22.1,65.0,2.2555,0.7682,24.923275,24.9665,2.0,-0.052557,-0.043438,-0.84643,1.443,-0.898987,1.399562,0.500575,"""active""","""real"""
"""IOTA_USDT""","""CELO_USDT""",2025-11-15 20:47:10 MSK,"""short""","""long""",370.3,243.5,0.1347,0.2051,24.939705,24.970925,2.0,-0.051648,-0.059828,1.1109,-1.6071,1.059252,-1.666928,-0.607676,"""active""","""real"""


In [ ]:
# db_manager.delete_pair_order('CELO_USDT', 'CRV_USDT')

In [ ]:
# db_manager.clear_old_data('tick_ob', column='time', expiration_time=192, units='hours')

In [ ]:
# db_manager.get_public_trades(exchange='bybit', token='ETH_USDT', return_type='polars')

#### Orders & trading_history

In [ ]:
db_manager.clear_table('trading_history')
db_manager.clear_table('current_orders')
# redis_order_manager.clear_orders_table()

In [ ]:
db_manager.delete_order('ADA_USDT', 'okx', 'linear')

In [ ]:
db_manager.place_order(token='ADA_USDT', exchange='okx', market_type='linear', order_type='market', order_side='buy',
                  qty=160, price=0.5927, usdt_amount=94.672, realized_pnl=-0.0025, leverage=2)

In [ ]:
db_manager.close_order(token='ADA_USDT', exchange='okx', market_type='linear', qty=160.0,
                       close_price=0.5927, close_usdt_amount=94.672, close_fee=-0.0025)

In [ ]:
current_pairs = db_manager.get_table('pairs')
current_pairs

In [ ]:
current_orders = db_manager.get_table('current_orders', df_type='polars')
current_orders

In [ ]:
trading_history = db_manager.get_table('trading_history', df_type='polars')
trading_history.select('created_at', 'closed_at', 'token', 'order_side', 'open_price', 'close_price', 
                       'open_usdt_amount', 'close_usdt_amount', 'qty', 'leverage', 'realized_pnl', 'profit')

In [ ]:
trading_history['profit'].sum()

In [ ]:
calculate_profit(open_price=0.4854, close_price=0.4849, n_coins=411.5, usdt_amount=None, side='long', fee_rate=0.00055)

#### Redis

In [ ]:
%load_ext autoreload
%autoreload 2

import redis
from redis.exceptions import ConnectionError
import polars as pl
import pandas as pd
from datetime import datetime
from decimal import Decimal
import orjson
import json

redis_client = redis.Redis(db=0, decode_responses=True)

from jaref_bot.db.redis_manager import RedisManager

redis_manager = RedisManager(db_name='orderbooks')

try: 
    redis_client.ping()
    print('Сервер Redis запущен')
except ConnectionError:
    print('Сервер Redis не отвечает')

In [ ]:
redis_client.flushdb()

In [ ]:
redis_client.keys()

In [ ]:
ts = redis_client.hget("orderbook:bybit:linear:ZBCN_USDT:update_time", 'cts')
datetime.fromtimestamp(ts)

In [ ]:
redis_client.hgetall("orderbook:bybit:linear:ZBCN_USDT")

In [ ]:
dd = redis_manager.get_orderbooks(5).drop('exchange', 'market_type').rename({'symbol': 'token'})
dd

In [ ]:
db_manager.add_orderbook_bulk(dd)

In [ ]:
db_manager.get_table('current_ob')